### Add Code Comment DocString to Python Function

In [40]:
import os 
import openai
import pprint
import inspect
from pathlib import Path

In [20]:
openai.api_key = os.environ["OPENAI_KEY"]

In [32]:
def ask_open_ai(prompt):
    res = openai.Completion.create(model= "text-davinci-003", ## "code-davinci-002",
                                   prompt = prompt,
                                   temperature = 0,
                                   top_p = 1.0,
                                   stop=['"""'],
                                   max_tokens = 256)
    return res
    
def create_prompt(code, lang="python", comment_format='"""'):
    prompt = f"{code}\n # A high quality {lang} docstring of the above {lang} function:\n {comment_format}"
    return prompt

def merge_docstring_and_function(function_string, docstring):
    split = function_string.split("\n")
    first_part, second_part = split[0], split[1:]
    merged_function = first_part + "\n" + '    """' + docstring + '    """' + "\n" + "\n".join(second_part)
    return merged_function

def get_all_functions(module):
    return [mem for mem in inspect.getmembers(module, inspect.isfunction)
         if mem[1].__module__ == module.__name__]


In [28]:
def hello(name=""):
    print(f"Hello {name}")

In [29]:
code = inspect.getsource(hello)
resp = ask_open_ai(create_prompt(code))
print( resp['choices'][0]['text'] )


This function prints a greeting to the given name.

Parameters:
name (str): The name of the person to greet. If no name is provided, a generic greeting is printed.

Returns:
None
 


In [43]:

## note you need to have a standard python file named (functions_without_docstrings.py) in 
## the same route path as this notebook in order to replace all the functions without docstrings with 
## a new file with code comment doc strings. 
import functions_without_docstrings
functions_with_prompts = []
functions_to_prompt = functions_without_docstrings
all_functions = get_all_functions(functions_without_docstrings)

In [44]:
### Create a prompt for each function in the functions_without_docstrings file and ###
### Run the prompts through the model and store the results in an output file called functions_without_docstrings_ ###
for func in all_functions:
    code = inspect.getsource(func[1])
    prompt = create_prompt(code)
    response = ask_open_ai(prompt)
    merged_code = merge_docstring_and_function(code, response["choices"][0]["text"])
    functions_with_prompts.append(merged_code)
    file_name = Path(functions_to_prompt.__file__).stem
    file_output = 'functions_with_docstring.py'
    print(f"Updating file {file_name} with doc strings to {file_output}")
    with open(f"{file_output}", "w") as f:
        f.write("\n\n".join(functions_with_prompts))

Updating file functions_without_docstrings with doc strings to functions_with_docstring.py
Updating file functions_without_docstrings with doc strings to functions_with_docstring.py
Updating file functions_without_docstrings with doc strings to functions_with_docstring.py
Updating file functions_without_docstrings with doc strings to functions_with_docstring.py
Updating file functions_without_docstrings with doc strings to functions_with_docstring.py
